In [3]:
import pandas as pd
import numpy as np
from influxdb import DataFrameClient
from dotenv import load_dotenv
import os
from fnmatch import fnmatch

In [2]:
load_dotenv()

True

In [ ]:
user = os.getenv('DB_USER')
password = os.getenv('DB_PWD')
host = os.getenv('DB_ADDR')
port = os.getenv('DB_PORT')
dbname = 'GasPriceMonitorization'
protocol = 'line'
client = DataFrameClient(host, port, user, password, dbname)

In [ ]:
measurement_gas = 'GasPrice'
measurement_crude = 'CrudeOil'

In [ ]:
def upload_all_gas():
    root = '../data'
    pattern = "gas_tmp-*.json"

    paths = []
    for path, subdirs, files in os.walk(root):
        for name in files:
            if fnmatch(name, pattern):
                paths.append(os.path.join(path, name))

    for path in paths:
        data = pd.read_json(path, orient='records')
        data.set_index('Timestamp', inplace=True)
        data.dropna(inplace=True)

        columns = data.columns
        columns = columns.drop(['Price'])

        client.write_points(data, measurement_gas, protocol=protocol, field_columns=['Price'], tag_columns=columns)

In [ ]:
def upload_all_crude():
    root = '../data'
    pattern = "crude_tmp-*.json"

    paths = []
    for path, subdirs, files in os.walk(root):
        for name in files:
            if fnmatch(name, pattern):
                paths.append(os.path.join(path, name))

    for path in paths:
        data = pd.read_json(path, orient='records')
        data.set_index('Date', inplace=True)
        data.dropna(inplace=True)

        client.write_points(data, measurement_crude, protocol=protocol, field_columns=['Closing'])

In [ ]:
upload_all_gas()
upload_all_crude()